# Übung 2: Hauptkomponentenanalyse

## 1. Implementierung der Hauptkomponentenanalyse

In [126]:
import pandas as pd
import numpy as np
import numpy.testing as npt
import json
import zipfile
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import cm

np.set_printoptions(suppress=True, linewidth=np.inf)

%reload_ext version_information
%version_information numpy, pandas, json, zipfile, plt, seaborn

Software versions
Python 3.7.4 64bit [GCC 7.3.0]
IPython 7.8.0
OS Linux 4.15.0 65 generic x86_64 with debian buster sid
numpy 1.17.2
pandas 0.25.1
json 2.0.9
zipfile The 'zipfile' distribution was not found and is required by the application
plt The 'plt' distribution was not found and is required by the application
seaborn 0.9.0
Mon Oct 28 11:09:37 2019 CET

In [127]:
'''
Attribute Information:
1. CRIM      per capita crime rate by town
2. ZN        proportion of residential land zoned for lots over 25,000 sq.ft.
3. INDUS     proportion of non-retail business acres per town
4. CHAS      Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
5. NOX       nitric oxides concentration (parts per 10 million)
6. RM        average number of rooms per dwelling
7. AGE       proportion of owner-occupied units built prior to 1940
8. DIS       weighted distances to five Boston employment centres
9. RAD       index of accessibility to radial highways
10. TAX      full-value property-tax rate per $10,000
11. PTRATIO  pupil-teacher ratio by town
12. B        1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town
13. LSTAT    % lower status of the population
14. MEDV     Median value of owner-occupied homes in $1000's
'''

url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/housing/housing.data'
cols = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'TGT']
boston = pd.read_csv(url, sep=' ', skipinitialspace=True, header=None, names=cols[0:13], index_col=False)
boston

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,396.90,9.14
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,392.83,4.03
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,394.63,2.94
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222.0,18.7,396.90,5.33
...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,0.06263,0.0,11.93,0,0.573,6.593,69.1,2.4786,1,273.0,21.0,391.99,9.67
502,0.04527,0.0,11.93,0,0.573,6.120,76.7,2.2875,1,273.0,21.0,396.90,9.08
503,0.06076,0.0,11.93,0,0.573,6.976,91.0,2.1675,1,273.0,21.0,396.90,5.64
504,0.10959,0.0,11.93,0,0.573,6.794,89.3,2.3889,1,273.0,21.0,393.45,6.48


In [128]:
def pca(data, r, whiten=True):
    n, d = data.shape
    
    if whiten:
        # Center
        data = data - data.mean(axis=0)
        npt.assert_almost_equal(data[:, 0].mean(), 0.0, decimal=10)

        # Normalize
        data = data / data.std(axis=0)
        npt.assert_almost_equal(data[:, 0].var(), 1.0, decimal=10)
    
    U, S, VT = np.linalg.svd(data)
    #print(U.shape, S.shape, VT.shape)
    
    data_whitened = data
    eigen_values = S[:r]**2 / (d - 1)
    eigen_vectors = VT
    principal_components = VT.T[:, :r]
    scores = (U[:, :r] @ np.diag(S[:r]))
    
    return data_whitened, eigen_values, principal_components, scores, eigen_vectors

In [129]:
def plot_visuals(data_whitened, scores):
    sns.set_style("dark")
    color1 = "blue"
    color2 = "red"
    fig, axes = plt.subplots(3,3, figsize=(15,10))
    sns.scatterplot(data_whitened[:, 0], data_whitened[:, 1], color=color1, palette ='deep', ax=axes[0, 0])
    sns.scatterplot(scores[:, 0], scores[:, 1], color=color2, palette ='deep', ax=axes[0, 0])
    axes[0, 0].set_title("XY-Ebene")
    axes[0, 0].set_xlabel("X")
    axes[0, 0].set_ylabel("Y")

    sns.scatterplot(data_whitened[:, 0], data_whitened[:, 2], color=color1, palette ='deep', ax=axes[0, 1])
    sns.scatterplot(scores[:, 0], scores[:, 2], color=color2, palette ='deep', ax=axes[0, 1])
    axes[0, 1].set_title("XZ-Ebene")
    axes[0, 1].set_xlabel("X")
    axes[0, 1].set_ylabel("Z")

    sns.scatterplot(data_whitened[:, 1], data_whitened[:, 2], color=color1, palette ='deep', ax=axes[0, 2])
    sns.scatterplot(scores[:, 1], scores[:, 2], color=color2, palette ='deep', ax=axes[0, 2])
    axes[0, 2].set_title("YZ-Ebene")
    axes[0, 2].set_xlabel("Y")
    axes[0, 2].set_ylabel("Z")

    sns.distplot(np.array(data_whitened[:, 0]), color=color1, ax=axes[1, 0], norm_hist=True, kde=False)
    sns.distplot(np.array(scores[:, 0]), color=color2, ax=axes[1, 0], norm_hist=True, kde=False)
    axes[1, 0].set_title("X-Verteilung")

    sns.distplot(np.array(data_whitened[:, 1]), color=color1, ax=axes[1, 1], norm_hist=True, kde=False)
    sns.distplot(np.array(scores[:, 1]), color=color2, ax=axes[1, 1], norm_hist=True, kde=False)
    axes[1, 1].set_title("Y-Verteilung")

    sns.distplot(np.array(data_whitened[:, 2]), color=color1, ax=axes[1, 2], norm_hist=True, kde=False)
    sns.distplot(np.array(scores[:, 2]), color=color2, ax=axes[1, 2], norm_hist=True, kde=False)
    axes[1, 2].set_title("Z-Verteilung")

    sns.kdeplot(np.array(data_whitened[:, 0]), color=color1, ax=axes[2, 0], shade=True)
    sns.kdeplot(np.array(scores[:, 0]), color=color2, ax=axes[2, 0], shade=True)
    axes[2, 0].set_title("X-Verteilung")

    sns.kdeplot(np.array(data_whitened[:, 1]), color=color1, ax=axes[2, 1], shade=True)
    sns.kdeplot(np.array(scores[:, 1]), color=color2, ax=axes[2, 1], shade=True)
    axes[2, 1].set_title("Y-Verteilung")

    sns.kdeplot(np.array(data_whitened[:, 2]), color=color1, ax=axes[2, 2], shade=True)
    sns.kdeplot(np.array(scores[:, 2]), color=color2, ax=axes[2, 2], shade=True)
    axes[2, 2].set_title("Z-Verteilung")

### Testen der PCA

In [130]:
data = [
[-10.0,  10.0, 2.5],
[ -9.0,   9.0, 0.45],
[ -8.0,   8.0, 0.4],
[ -7.0,   7.0, 1.35],
[ -6.0,   6.0, 0.3],
[ -5.0,   5.0, 0.25],
[ -4.0,   4.0, 0.2],
[ -3.0,   3.0, 0.15],
[ -2.0,   2.0, 0.1],
[ -1.0,   1.0, 0.05],
[  0.0,   0.0, 0.0],
[  1.0,  -1.0, 0.05],
[  2.0,  -2.0, 0.1],
[  3.0,  -3.0, 0.15],
[  4.0,  -4.0, 4.2],
[  5.0,  -5.0, 0.25],
[  6.0,  -6.0, 0.3],
[  7.0,  -7.0, 0.35],
[  8.0,  -8.0, 0.4],
[  9.0,  -9.0, 1.45],
[ 10.0, -10.0, 0.5],

[-10.0, -10.0, 0.5],
[ -9.0,  -9.0, 0.45],
[ -8.0,  -8.0, 0.4],
[ -7.0,  -7.0, 0.35],
[ -6.0,  -6.0, 0.3],
[ -5.0,  -5.0, 0.25],
[ -4.0,  -4.0, 3.2],
[ -3.0,  -3.0, 0.15],
[ -2.0,  -2.0, 0.1],
[ -1.0,  -1.0, 2.05],
[  0.0,  -0.0, 0.0],
[  1.0,   1.0, 0.05],
[  2.0,   2.0, 0.1],
[  3.0,   3.0, 0.15],
[  4.0,   4.0, 0.2],
[  5.0,   5.0, 0.25],
[  6.0,   6.0, 1.3],
[  7.0,   7.0, 0.35],
[  8.0,   8.0, 0.4],
[  9.0,   9.0, 0.45],
[ 10.0,  10.0, 0.5]]
data = np.asarray(data)
data_whitened, eigen_values, principal_components, scores, eigen_vectors = pca(data, 3, whiten=True)

print(data_whitened.var(axis=0))
print(scores.var(axis=0))
#print(eigen_values)
print(eigen_vectors)
#plot_visuals(data_whitened, scores)

[1. 1. 1.]
[1.05239502 1.         0.94760498]
[[ 0.60633906  0.36380344 -0.70710678]
 [-0.51449576  0.85749293 -0.        ]
 [ 0.60633906  0.36380344  0.70710678]]


In [131]:
x = np.random.randint(100, size=100)
y = np.random.randint(100, size=100)
z = np.random.randint(100, size=100)
data = np.asarray([x, y, z]).T
data_whitened, eigen_values, principal_components, scores, eigen_vectors = pca(data, 3, whiten=True)

print(scores.var(axis=0))
print(eigen_values)
#plot_visuals(data_whitened, scores)

[1.27622145 1.01515294 0.70862561]
[63.81107246 50.75764716 35.43128039]


### PCA Boston Housing-Test

In [132]:
boston_array = np.asarray(boston)
data_whitened, eigen_values, principal_components, scores, eigen_vectors = pca(boston_array, len(boston_array.T))
eigen_values_df = pd.DataFrame(columns=["Eigenvalues", "Anteilige Varianz", "Kumulative Varianz"])
eigen_values_df.iloc[:, 0] = eigen_values
eigen_values_df.iloc[:, 1] = np.var(scores, axis=0) / np.var(scores, axis=0).sum()
eigen_values_df.iloc[:, 2] = np.cumsum(eigen_values_df.iloc[:, 1]) * 100
eigen_values_df

,Eigenvalues,Anteilige Varianz,Kumulative Varianz
0,258.348792,0.471296,47.129606
1,60.436434,0.110252,58.154800
2,52.397003,0.095586,67.713389
3,36.161084,0.065967,74.310121
4,35.201405,0.064217,80.731782
5,27.720669,0.050570,85.788760
6,22.574182,0.041181,89.906884
7,16.702103,0.030469,92.953786
8,11.677777,0.021303,95.084120
9,9.286695,0.016941,96.778257


10%:  8 Dimensionen

 5%:  9 Dimensionen

 1%: 12 Dimensionen

In [136]:
scores_df = pd.DataFrame(scores[:, :3])
merged_df = pd.concat([scores_df, boston], axis=1, sort="False")
corr = merged_df.corr()
cmap = cm.get_cmap('Greens', 256)
corr.style.background_gradient(cmap=cmap, axis=None).set_precision(2)

,0,1,2,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,1,6.7e-17,-3e-17,0.62,-0.63,0.86,0.012,0.85,-0.47,0.78,-0.8,0.79,0.84,0.51,-0.5,0.77
1,6.7e-17,1,-4.6e-16,-0.38,-0.39,0.13,0.54,0.26,0.18,0.37,-0.42,-0.33,-0.29,-0.37,0.29,-0.089
2,-3e-17,-4.6e-16,1,0.27,0.33,-0.018,0.32,0.13,0.66,-0.02,-0.055,0.32,0.25,-0.36,-0.33,-0.3
CRIM,0.62,-0.38,0.27,1,-0.2,0.41,-0.056,0.42,-0.22,0.35,-0.38,0.63,0.58,0.29,-0.39,0.46
ZN,-0.63,-0.39,0.33,-0.2,1,-0.53,-0.043,-0.52,0.31,-0.57,0.66,-0.31,-0.31,-0.39,0.18,-0.41
INDUS,0.86,0.13,-0.018,0.41,-0.53,1,0.063,0.76,-0.39,0.64,-0.71,0.6,0.72,0.38,-0.36,0.6
CHAS,0.012,0.54,0.32,-0.056,-0.043,0.063,1,0.091,0.091,0.087,-0.099,-0.0074,-0.036,-0.12,0.049,-0.054
NOX,0.85,0.26,0.13,0.42,-0.52,0.76,0.091,1,-0.3,0.73,-0.77,0.61,0.67,0.19,-0.38,0.59
RM,-0.47,0.18,0.66,-0.22,0.31,-0.39,0.091,-0.3,1,-0.24,0.21,-0.21,-0.29,-0.36,0.13,-0.61
AGE,0.78,0.37,-0.02,0.35,-0.57,0.64,0.087,0.73,-0.24,1,-0.75,0.46,0.51,0.26,-0.27,0.6


## 2. Eigengesichter

In [ ]:
import pandas as pd
import numpy as np
import tarfile, requests, wget
import os
import skimage as ski
from sklearn.decomposition import PCA
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt

np.set_printoptions(suppress=True, linewidth=np.inf)

!pip install version_information
%reload_ext version_information
%version_information numpy, pandas, tarfile, requests, wget, os, skimage, matplotlib

In [ ]:
filename = "lfw-funneled"
dirname = "lfw_funneled"
url = "http://vis-www.cs.umass.edu/lfw/"+filename+".tgz"
tar_file_name = "../data/"+filename+".tgz"
untar_file_name = "../data/"
wget.download(url, tar_file_name) if not os.path.isfile(tar_file_name) else print("File already downloaded.")
tarfile.open(tar_file_name, "r:gz").extractall(untar_file_name) if not os.path.isdir(untar_file_name+dirname) else print("File already extracted.")

In [ ]:
def process_img(img):
    # crop, scale, flatten
    y_l, y_u = 90, 185
    x_l, x_u = 80, 170
    img_cropped = img[y_l:y_u, x_l:x_u]
    img_scaled = ski.transform.resize(img_cropped, (32, 32), anti_aliasing=True)
    img_flatten = img_scaled.flatten()
    return img_flatten

In [ ]:
train = []
validate = []
for path, dirs, files in os.walk(untar_file_name):
    if len(files) >= 70:
        name = path.split("/")[-1]
        for i in range(len(files) - 1):
            img = ski.io.imread(os.path.join(path, files[i]), as_gray=True)
            img_processed = process_img(img)
            train.append(img_processed)
        img = ski.io.imread(os.path.join(path, files[-1]), as_gray=True)
        validate.append([process_img(img), name])

train = np.asarray(train)
validate = np.asarray(validate)
train.shape, validate.shape

In [ ]:
dim = 1024
data_whitened, eigen_values, principal_components, scores, eigen_vectors = pca(train, dim)
print(data_whitened.shape, eigen_values.shape, principal_components.shape, scores.shape, eigen_vectors.shape)
print(scores.var(axis=0))
print(eigen_values)

In [ ]:
fig, axes = plt.subplots(1,2, figsize=(20,10))
sns.scatterplot(x=np.arange(0,150), y=eigen_values[0:150], color='blue', ax=axes[0])
sns.scatterplot(x=np.arange(0,150), y=scores.var(axis=0)[0:150], color='red', ax=axes[1])
axes[0].grid(b=True, which='major', color='gray', linewidth=0.5)
axes[1].grid(b=True, which='major', color='gray', linewidth=0.5)
axes[0].set_title("First 150 Eigenvalues")
axes[1].set_title("Variance of first 150 scores")

In [ ]:
ncols, nrows = 4, 3
fig = plt.figure(figsize=(15,10))
spec = gridspec.GridSpec(ncols=ncols, nrows=nrows, figure=fig)
for i in range(nrows):
    for j in range(ncols):
        ax = fig.add_subplot(spec[i, j])
        img = eigen_vectors[i * ncols + j].reshape(32, 32)
        min_val = img.min()
        img -= min_val
        max_val = img.max()
        img *= (255 / max_val)
        ski.io.imshow(img, ax=ax, cmap="gray")

In [ ]:
fig = plt.figure(figsize=(15,10))
spec = gridspec.GridSpec(ncols=7, nrows=1, figure=fig)
for i in range(7):
        ax = fig.add_subplot(spec[0, i])
        img = validate[i, 0] - train.mean(axis=0)
        img = img.reshape(32, 32)
        min_val = img.min()
        img -= min_val
        max_val = img.max()
        img *= (255 / max_val)
        ski.io.imshow(img, ax=ax, cmap="gray")

In [ ]:
train_m = train - train.mean(axis=0)
train_p = train_m @ eigen_vectors[:, :7]

fig = plt.figure(figsize=(30,20))
spec = gridspec.GridSpec(ncols=2, nrows=7, figure=fig)
for i in range(7):
    validate_img = validate[i, 0] - train.mean(axis=0)
    validate_img_p = validate_img @ eigen_vectors[:, :7]

    min_index = 0
    min_val = np.inf
    for j in range(1281):
        min_tmp = np.linalg.norm(train_p[j] - validate_img_p)
        if min_tmp < min_val:
            min_val = min_tmp
            min_index = j
    print(min_val, min_index)
    ax1 = fig.add_subplot(spec[i, 0])
    ax2 = fig.add_subplot(spec[i, 1])
    ax1.set_title("Validate image")
    ax2.set_title("Image with smallest euclidian distance")
    validate_img = validate_img.reshape(32, 32)
    img = train[min_index].reshape(32, 32)
    ski.io.imshow(validate_img, cmap="gray", ax=ax1)
    ski.io.imshow(img, cmap="gray", ax=ax2)